# Character-based N-gram Modeling Predicts Language Groups and Evolution
Koray Akduman | March 3, 2024

# Introduction
The study of language evolution and grouping is a pivotal area of research that delves into the origins, development, and classification of languages. Understanding unlocks the mysteries of human communication, social dynamic, and even biological evolution (Dunbar, 1997). Languages serve as a mirror reflecting the socio-political changes, migratory patterns, and cultural interactions of different communities. By tracing the genealogy and transformation of languages, researchers can piece together the puzzle of human civilization and its diverse manifestations.

The classification and analysis of languages have traditionally relied on historical linguistics, which employs comparative methods to identify similarities and differences among languages. However, this approach is often constrained by the availability of historical records and the subjective interpretation of linguistic features. Machine learning offers a data-driven, computational, and unbiased approach to studying language evolution and grouping. Machine learning algorithms can process vast amounts of linguistic data, detect patterns and correlations, and provide insights that are sometimes beyond the reach of an initial manual analysis. This technological leap has the potential to refine our understanding of language relationships and to uncover previously unrecognized connections among languages. Further, machine learning in the field of linguistic analysis can produce better models for capturing the similarities in language--allowing for multi-lingual data analysis to sustain the growing demand in data by popular large language models (LLMs) (Seki, 2020).

While machine learning is most often used for translation models in the field of linguistics, they can also be particularly useful in addressing the challenges of language classification and evolution. For instance, models can aid in the identification of language families and subgroups by analyzing phonetic, morphological, and syntactic features. By leveraging unsupervised learning techniques, such as clustering algorithms, researchers can discover natural groupings of languages based on their similarities. Moreover, machine learning models can be trained to predict the historical changes in languages, providing a dynamic view of linguistic evolution. This can help in reconstructing proto-languages and in understanding the mechanisms of language change over time (Caselli et al., 2022).

In this paper, as a proof-of-concept, we implement a simple character-based n-gram approach that successfully predicts commonly recognized language families (e.g. "West Germanic," "Romance," etc.) in a completely unsupervised manner, using k-means clustering and Uniform Manifold Approximation and Projection (UMAP) to visualize the vectors. Further, our character-based n-gram approach classifies languages like "Old German" and "German" into the same language group, demonstrating an ability to trace backwards in time to determine the evolution of language through machine learning. Our results also show that a character-based n-gram approach is able to find the relationships between different language groups, predicting, for example, that West Germanic and North Germanic languages are similar.


# Background
Machine learning has been demonstrated to capture various complex facets of human language, including semantic meaning across different languages (Seki, 2020). Seki compares models that use word and paragraph features to determine the semantic meaning of a text, with unexplored models that pay special attention to the intermediate states of translation models when translating the given text. Seki finds that models focusing on intermediate states of translation models outcompete those focusing solely on words and paragraphs, when computing similarity between cross-lingual texts. These findings suggest that much of the language information is contained in computerized representations of language rather than solely the text, and that language can be represented computationally.

 Character-based n-grams refer to breaking a larger string into a series of strings ("n-grams") that are n-long in length, preserving the order of the individual characters within the string (Figure 1). For machine learning, the frequency of these n-grams are represented in a unit vector (Figure 1). This unit vector, then, demonstrates the relationship between individual characters in that language. When differentiating between cross-lingual texts, Previous researchers have demonstrated the capabilities of relatively simple character-based n-gram models to have high-success (90+%) at differentiating between similar languages (Cianflone, 2017). Further research has found that character-based n-grams successfully classify articles into different languages with above 99% accuracy, and categories of articles with 80% accuracy (Cavnar & Trenkle, 2001). 

N-grams have found great success in non-linguistic fields, too. In biology, a k-mer refers to a string of consecutive base pairs k-long in length. K-mers have been shown to capture the relationship between base pairs in a genetic sequence. The field of genetic engineering attribution (GEA) explores the biosecurity problem of attributing genetically engineered sequences to the lab that created them. Alley et al. (2020) finds that k-mers can significantly improve success in identifying labs by recognizing functional motifs in genetically-engineered plasmids. Akduman (2023) further explores k-mers in genetic engineering, and finds that a simple logistic regression model, using plasmids vectorized into k-mer proportions, can predict the originating lab of an engineered sequence out of over 2,000 labs with an approximately 45% success, and places the originating lab in the top-10 most likely labs with an approximately 75% success.

However, character-based n-grams have not yet been explored to map similarities between languages to determine their capacity for clustering similar languages, finding language groups, and determining the relationship between language families. This study presents a proof-of-concept application of character-based n-grams to visualizing language simiularity.

# Methods
We analyzed the [Tatoeba sentences dataset](https://downloads.tatoeba.org/exports/sentences.tar.bz2), a dataset available by the CC-BY License containing 11,879,690 simple sentences in 423 languages. Of these languages, 162 have greater than 500 sentences. We focused on these languages to minimize our noise. The Tatoeba dataset was chosen after careful consideration of many other datasets. We initially analyzed the Wikipedia page data dataset, but determined that strange formatting in Wikipedia pages significantly increased noise in our data (e.g. section headings were often untranslated, determining the language was difficult, and there were lots of symbols), along with having limited languages available. We tried translating the dataset to other languages, but ran into problems with translation API rate limits. Further, it is likely that these translation APIs bias our data slightly by preferring some words that native speakers may not use as often. The Tatoeba sentences dataset provides sufficient data in a significant number of languages, making it optimal for our study.

We broke down these sentences from each language into n-mers of length 2-4 using Scikit-learn's Term Frequency - Inverse Document Frequency (TF-IDF) vectorizer with n-gram lengths specified as 2-4, which outputs unit vectors with each dimension corresponding to an n-gram (Example: Figure 1). 

Our justification for 2-4 comes partially from Cianflone (2017) and partially from hardware constraints. Cianflone used n-grams of length 1-8, with increasing accuracy as n-gram length increases until n=8. However, memory usage explodes as n increases, as vector dimensions increase exponentially with n. Our system would consistently crash at length 1-4 and 3-5, leaving 2-4 as the optimal n-gram range that covers a range of n-grams while preferring larger n-grams that are shown to be superior indicators of language similarity (Cianflone 2017). At n=2-4, we noticed that our system was using 8GB of memory and 22GB of swap. While certainly unstable, it would generally complete processing. We also dropped n-grams that occurred in less than 1% of documents, as our goal was to find similarities between languages and this would likely create noise, in addition to increasing dimensionality and future computation time.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer for character-based 2-4-grams
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4))
sentence = ["hello"]

# Fit and transform the sentence
X = vectorizer.fit_transform(sentence)

# Print the 4-grams & corresponding TF-IDF values
feature_names = vectorizer.get_feature_names_out()
print("4-grams:", feature_names)
print("TF-IDF values:", X.toarray())

4-grams: ['el' 'ell' 'ello' 'he' 'hel' 'hell' 'll' 'llo' 'lo']
TF-IDF values: [[0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333 0.33333333 0.33333333]]


**Figure 1: N-gram calculation using TF-IDF vectorizer.** The word "hello" is split into its 2-grams, 3-grams, and 4-grams, then normalized to a unit vector.

After computing the n-gram vectors for each sentence, we then averaged the vectors for each language. Again due to hardware constraints, we used a memory-reduced method of averaging through an array that would process averages in batches of 1000 sentences. We then performed k-means clustering to cluster the languages into 14 groups. This number was calibrated to reflect maximal variation in language groups without splitting up the arbitrarily selected "West Germanic" languages family.

We then analyzed the identities of the groups selected in the k-means clusters, and created a UMAP of our data to visualize it in two dimensions.

# Results

Our results demonstrate that character-based n-grams are able to be used as feature vectors for k-means clustering and UMAP. These projections and clusters identify similar languages or language families in the same cluster, and similar languages are close to each other in the UMAP analysis. These findings demonstrate that character-based n-grams of n=2-4 are able to sufficiently capture many relationships between languages. It also affirms the idea of classifying languages based on their origins, where unsupervised machine learning models are able to find similar groups to the ones that we have classified based on their origins.

Our results show clusters for Romance languages (cluster 1), West Germanic languages (cluster 7), North Germanic languages (cluster 4), and many other similar language groups (Table 1). Just as importantly, our classification determines relationships between languages and their origins. For example, French and Middle French are both in cluster 1, English, Middle English, and Old English are both in cluster 7, and Upper Sorbian and Lower Sorbian are both in cluster 11--likely as a result of diverging from each other at some point in time.

Our data also show that character-based n-gram characterization is insufficient for many ideographic-based languages. For example, Mandarin Chinese and Egyptian Arabic are both in cluster 3, despite having significantly different origins. This improper grouping therefore demonstrates an unusually large Cluster 3 in the histogram,  reflecting flaws with character-based analysis between languages that do not share characters (Figure 2). In this case, our approach simply groups non-alphabetical languages into their own clusters and distances them from alphabetical languages in the UMAP (Figure 3), while perhaps not capturing the similarities and differences between ideographic languages.

| Cluster | Languages |
|---|---|
| 0 | Tatar, Kazakh, Ossetian, Kalmyk, Chuvash, Udmurt, Kyrgyz, Chukchi, Bashkir |
| 1 | French, Spanish, Italian, Esperanto, Portuguese, Latin, Romanian, Latvian, Catalan, Middle French, Galician, Lojban, Interlingua, Romansh, Ido, Asturian, Occitan, Interlingue, Novial, Ladin, Piedmontese, Ladino, Chavacano, Picard |
| 2 | Turkish, Hungarian, Breton, Uzbek, Quenya, Basque, Irish, Klingon, Scottish Gaelic, Ottoman Turkish, Unknown, Sindarin, Azerbaijani, Turkmen, Northern Sami, Welsh, Northern Kurdish (Kurmancî), Kabyle, Berber, Haitian Creole, Cornish, Nahuatl, Kotava, Seychellois Creole, Navajo |
| 3 | Mandarin Chinese, Japanese, Korean, Hebrew, Greek, Arabic, Egyptian Arabic, Cantonese, Shanghainese, Georgian, Urdu, Sanskrit, Yiddish, Uyghur, Persian, Iraqi Arabic, Armenian, Hindi, Bengali, Thai, Literary Chinese, Malayalam, Telugu, Khmer, Tibetan, Punjabi (Western), Ancient Greek, Lao, Nepali, Marathi, Gujarati, Punjabi (Eastern), Sumerian, Cherokee, Algerian Arabic, Tamil |
| 4 | Swedish, Norwegian Nynorsk, Norwegian Bokmål, Icelandic, Faroese, Danish |
| 5 | Mongolian, Tajik |
| 6 | Russian, Bulgarian, Ukrainian, Belarusian, Serbian, Old East Slavic, Macedonian, Erzya, Abkhaz |
| 7 | German, English, Dutch, Frisian, Afrikaans, CycL, Low German (Low Saxon), Luxembourgish, Scots, Old English, Kölsch, Middle English, Swiss German |
| 8 | Vietnamese |
| 9 | Tagalog, Hiligaynon |
| 10 | Quechua |
| 11 | Czech, Polish, Albanian, Slovak, Bosnian, Croatian, Slovenian, Volapük, Maltese, Upper Sorbian, Lower Sorbian, Ewe, Romani |
| 12 | Indonesian, Finnish, Swahili, Malay, Estonian, Lithuanian, Chamorro, Sicilian, Ainu, Toki Pona, Javanese, Tok Pisin, Maori, Old Tupi, Guarani, Malagasy, Xhosa, Old Prussian, Malay (Vernacular), Lakota |
| 13 | Lingala |

**Table 1: All languages grouped into their respective clusters based on k-means clustering.** (k=14) Many groups can be identified to resemble well-known language groups. Cluster 1 contains the Romance languages French, Spanish, Italian, Portuguese, and Latin, among others. Cluster 4 contains the North Germanic languages Swedish, Norwegian Nynorsk, Norwegian Bokmål, Icelandic, Faroese, and Danish. Cluster 7 contains the West Germanic languages German, English, Dutch, Frisian, and Afrikaans, among others.

![Figure 2: Cluster Distribution](image.png)

**Figure 2: Distribution of k-mean clusters.** Cluster 3, largely representing character-based languages, has the greatest number of members

A UMAP of the results verifies the similarities between language groups. While k-means clusters demonstrate the ability to classify languages into their specific language groups, the UMAP permits us to view the relationships between different language groups. We notice, for example, that West Germanic languages and North Germanic languages are next to each other in the UMAP of our data. This closeness may be a result of similar origins, with both North Germanic and West Germanic languages being similar language families.

![Figure 3: UMAP](image-1.png)

**Figure 3: UMAP of languages with >250 sentences.** The upper-right section of the left graph represents alphabetical languages, which can be further analyzed to determine the relationship between languages in a non-arbitrary manner. The right graph shows a zoomed in version of the first graph, reflecting the similarities in languages in this region. Similar languages (as determined by k-means clustering) appear close to each other, while dissimilar languages appear far from each other. The brown dots represent West Germanic languages, baby blue represents Romance languages, and red represents North Germanic languages. The 20 languages with the most sentences in the dataset are labeled.

# Conclusion
Our project shows a promising proof-of-concept for language characterization using character-based n-gram vectorization with limited computational resources. We are successfully able to predict existing language families using n-grams of 2-4 in length, meant to portray the relationship between individual characters. We demonstrate, therefore, that language groups are often identified through the localized relationships that individual characters have with other characters in the language. Our ability to use n-grams to predict language families indicates that these character relationships are additionally largely maintained as a language progresses (e.g. indicated by Old English, Middle English, and English all being in the same cluster). We show, additionally, that the character relationships slowly evolve over large periods of time. Similar language families, like North Germanic and West Germanic, are in clearly distinct portions of the UMAP visualization, yet these groups are also close in distance.

Moreover, our research highlights the potential of unsupervised machine learning models in classifying languages based on their origins, without the need for predefined labels. The ability of our model to cluster languages into groups that align with commonly recognized language families underscores the effectiveness of character-based n-gram modeling in linguistic analysis.

This project also served as an introduction to machine learning given resource constraints. We did all data analysis and processing on an 8GB MacBook Air, which was wholly insufficient, primarily for memory reasons. When all of the dataset was loaded into memory the computer would regularly show python using 30+GB of memory, with the computer using 22GB of swap storage. The system also crashed due to lack of application memory on many occasions, resulting in the save statements at every step that now exist in main.ipynb. If this project was to be repeated in the future as more than a proof of concept, we would use a virtual server instance with heightened memory resources, enabling more complex n-gram data analysis.

# References
Akduman, K. (2023). Unpublished Data: Genetic Attribution Through K-mer Proportions.

Alley, E. C., Turpin, M., Liu, A. B., Kulp-McDowall, T., Swett, J., Edison, R., Von Stetina, S. E., Church, G. M., & Esvelt, K. M. (2020). A machine learning toolkit for genetic engineering attribution to facilitate biosecurity. Nature Communications, 11(1), 6293. https://doi.org/10.1038/s41467-020-19612-0

Caselli, N., Occhino, C., Artacho, B., Savakis, A., & Dye, M. (2022). Perceptual optimization of language: Evidence from american sign language. Cognition, 224, 105040. https://doi.org/10.1016/j.cognition.2022.105040

Cavnar, W., & Trenkle, J. (2001). N-Gram-Based Text Categorization. Proceedings of the Third Annual Symposium on Document Analysis and Information Retrieval. https://www.researchgate.net/publication/2375544_N-Gram-Based_Text_Categorization/

Cianflone, A., & Kosseim, L. (2017, August 11). N-gram and neural language models for discriminating similar languages. arXiv.Org. https://arxiv.org/abs/1708.03421v1

Dunbar, R. I. M. (1997). Groups, gossip, and the evolution of language. In A. Schmitt, K. Atzwanger, K. Grammer, & K. Schäfer (Eds.), New Aspects of Human Ethology (pp. 77–89). Springer US. https://doi.org/10.1007/978-0-585-34289-4_5

Seki, K. (2021). Cross-lingual text similarity exploiting neural machine translation models. Journal of Information Science, 47(3), 404–418. https://doi.org/10.1177/0165551520912676

Tatoeba. (2024). Sentences [dataset]. https://tatoeba.org/en/downloads
